In [20]:
import pickle
import pandas as pd
import nltk
import nltk
nltk.download('stopwords')
print('------------')
nltk.download('punkt')
print('------------')
nltk.download('wordnet')
from nltk.corpus import stopwords

from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
#Read the csv file from drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
------------
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
------------
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
pickle_id = "1zj05sWx5oUfQ1TCb2BUdL6UjHi2rfM3k"
downloaded = drive.CreateFile({'id':pickle_id}) 
downloaded.GetContentFile('best_lrc.pickle')
with open('best_lrc.pickle', 'rb') as data:
    lrc_model = pickle.load(data)

In [0]:
#TFID Object
pickle_id = "1OmkdxXkvCw67pg4DjMMWaazBNKNi_F8j"
downloaded = drive.CreateFile({'id':pickle_id}) 
downloaded.GetContentFile('tfidf.pickle')
with open('tfidf.pickle', 'rb') as data:
    tfidf = pickle.load(data)

In [0]:
label_codes = {
    'none': 0,
    'soft': 1,
    'tech': 2,
}

In [0]:
punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))

def create_features_from_text(text):
    
    # Dataframe creation
    lemmatized_text_list = []
    df = pd.DataFrame(columns=['Content'])
    df.loc[0] = text
    df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
    df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
    df['Content_Parsed_3'] = df['Content_Parsed_2']
    for punct_sign in punctuation_signs:
        df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
    df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    text = df.loc[0]['Content_Parsed_4']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = " ".join(lemmatized_list)    
    lemmatized_text_list.append(lemmatized_text)
    df['Content_Parsed_5'] = lemmatized_text_list
    df['Content_Parsed_6'] = df['Content_Parsed_5']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
    df = df['Content_Parsed_6']
    df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
    
    # TF-IDF
    features = tfidf.transform(df).toarray()
    
    return features

In [0]:
def get_category_name(category_id):
    for category, id_ in label_codes.items():    
        if id_ == category_id:
            return category

In [29]:
lrc_model

LogisticRegression(C=0.6444444444444444, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='multinomial', n_jobs=None,
                   penalty='l2', random_state=8, solver='sag', tol=0.0001,
                   verbose=0, warm_start=False)

In [0]:
def predict_from_text(text):
    
    # Predict using the input model
    prediction_lrc = lrc_model.predict(create_features_from_text(text))[0]
    prediction_lrc_proba = lrc_model.predict_proba(create_features_from_text(text))[0]
    
    # Return result
    category_lrc = get_category_name(prediction_lrc)
    
    print("The predicted category using the LRC model is %s." %(category_lrc) )
    print("The conditional probability is: %a" %(prediction_lrc_proba.max()*100))

In [33]:
text="Kreativität und schnelle Auffassungsgabe"
predict_from_text(text)

The predicted category using the LRC model is soft.
The conditional probability is: 89.9046668593414



**Conclusion for result**

{
  "label":"soft",
  "text":"Kreativität und schnelle Auffassungsgabe"
}

*Multinomial Logistic Regressio*n is the best model for our case of text classification
the model predict the text is Soft with conditional probability  90%